# Geospatial aggregates for poverty comparison

In support of the ELCPV project (PXXXXXX), GOST are calculating admin 2 level summaries of a series of risk, climate, and environmental data. The focus will be on the following datasets

| Data layer | Source | Link | Notes |
| :--- | :--- | :--- | :--- |
| Floods | Fathom | [DDH](https://datacatalog.worldbank.org/int/search/dataset/0021728/Global-Flood-Hazard--FATHOM-) | GFDRR are working on acquiring an updated dataset, need funding |
| Drought | **TBD**  |  | One of the candidates is utilizing TerraClimate's Precipitation, Potential Evapotranspiration and Runoff to derive the SPI and SPEI as proxy for Meteorological Drought and SRI for the Hydrological Drought for period 1958-2022 |
| Heatwave | DDP | [Github](https://github.com/datapartnership/heatwaves) | The Data Partnership have developed tools for evaluating heat waves, but it is a work in progress |
| Cyclone | [NOAA IBTrACS](https://www.ncei.noaa.gov/products/international-best-track-archive) | S3 bucket: [IBTrACS](s3://wbgdecinternal-ntl/climate/data/cyclone/ibtracs) | Available in points and lines shapefile, netCDF and CSV. From 25 Oct 1842 to 7 Sep 2023 |
| Wildfire | **TBD** |  | Global Fire Emissions Database [GFED](https://www.globalfiredata.org/) as one of potential data |
| Temperature | CCKP | [Climate Change Knowledge Portal](https://climateknowledgeportal.worldbank.org/download-data) | The CCKP are working on an updated, 1/4 degree resolution dataset that we will leverage |
| Precipitation | CCKP | [Climate Change Knowledge Portal](https://climateknowledgeportal.worldbank.org/download-data) | The CCKP are working on an updated, 1/4 degree resolution dataset that we will leverage |
| Sea-level rise | CCKP | CCKP's [3rd party site](https://data.cckp.adamplatform.eu/InundationMaps/) | The SLR projection available for all SSP's Scenario |
| Population | WorldPop | [WorldPop](https://www.worldpop.org/) | We hope to use the soon-to-be released Global WorldPop v2 |
| Schools | OSM |  | Extracted via [HOT Export Tool](https://export.hotosm.org/en/v3/) |
| Health facilities | OSM |  | [HOT Export Tool](https://export.hotosm.org/en/v3/) |
| Road density | OSM |  | [HOT Export Tool](https://export.hotosm.org/en/v3/) |
| Dams | **TBD** | [DDH](https://datacatalog.worldbank.org/search/dataset/0038478) | Global Dams Database available in DDH |

In [1]:
import sys, os, importlib, math, multiprocessing
import rasterio, geojson

import pandas as pd
import geopandas as gpd
import numpy as np

from h3 import h3
from tqdm import tqdm
from shapely.geometry import Polygon

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.ntlMisc as ntl
import GOSTRocks.mapMisc as mapMisc
from GOSTRocks.misc import tPrint

sys.path.append("../../../src")
import h3_helper
import country_zonal

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
